In [67]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import os
import json 
import unicodedata
import re
import random


import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import joblib
USE_CUDA = torch.cuda.is_available()
device=torch.device('cpu')
device = torch.device("cuda" if USE_CUDA else "cpu")
import yaml
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
from TaskModelling import TaskModelling
from QAModels.Question_Answering import QuestionAnswering
from GetTasks import GetTasks
from tqdm import notebook
import joblib
import datetime
import os
from colorama import Fore, Back, Style
from termcolor import colored
from IPython.display import Markdown
import string
import scipy.spatial.distance as distance

## Corona Side

In [7]:
Task_Path='Tasks'
Embedding_Path='C:/Users/mkork/Desktop/BertTry/glove.840B.300d.txt'

In [8]:
embeddings=TaskModelling(Embedding_Path,Task_Path)

In [9]:
tasks=GetTasks.get_tasks(Task_Path)

In [10]:
def read_dict_sum():
    def merge_files(path_d, path_s):
        all_dicts={}
        all_sums={}
        for path_d, path_s in zip(path_d, path_s):
            dicts=joblib.load(path_d)
            sums=joblib.load(path_s)
            all_dicts={**all_dicts, **dicts}
            all_sums={**all_sums, **sums}
        return all_dicts, all_sums
    datas='QAResults'
    files=os.listdir(os.path.join(datas))
    dicts=[]
    summaries=[]
    for file in files:
        if file.endswith('.dic'):
            dicts.append(os.path.join(datas,file))
        if file.endswith('.sum'):
            summaries.append(os.path.join(datas,file))
    return merge_files(dicts,summaries)

In [11]:
all_dicts, all_sums=read_dict_sum()

In [12]:
task_questions=joblib.load('tasks/task_questions')

In [13]:
new_embeds=embeddings.fit()

In [14]:
def find_task_Summary(scores):
    text='The summary of 3500 paper that we know is'+os.linesep
    #task=int(scores[0][-1])
    text= text+all_sums[scores]
    return scores, text 

In [15]:
def find_task_ques(scores):
    text='My First answer is:'
    #task=int(scores[0][-1])
    rangeS=2
    for i in range(rangeS):
        text= text+all_dicts[scores].iloc[i,3]
        if i<rangeS-1:
            text= text+os.linesep+'Another Answer is:'
    return scores, text 

In [16]:
def End_conv():
    time=int(str(datetime.datetime.now().time())[:2])
    if time>18 or time<6:
        response='Have a Good evening'
    else:
        response='Have a Good Day'
    return response
  

In [17]:
Cov_list=['COVID-19', 'SARS-CoV-2', '2019-nCov', 'SARS Coronavirus 2' ,'2019 Novel Coronavirus','2019nCoV','COVID19','SARSCoV2',
         'SARSCoronavirus2','2019NovelCoronavirus']


In [73]:
Related_word=Cov_list+['Virus','transmission','incubate', 'incubation','weather','summer','virus','Corona','corona','prevent', 'mask','disinfectant']

In [18]:
def check_distanceq(best,query):
    idx=best[0]
    query=query.translate(str.maketrans('', '', string.punctuation))
    querys=embeddings.get_text(query)
    distances={}
    for query in querys:
        if query in Cov_list:
            query='Coronavirus'
        query_embed=embeddings.embedding_index[query]
        for i,sques in enumerate(task_questions[idx]):
            sques=sques.translate(str.maketrans('', '', string.punctuation))
            quests=embeddings.get_text(sques)
            for qw in quests:
                if qw in Cov_list:
                    qw='Coronavirus'
                value=embeddings.embedding_index[qw]
                sim=1-distance.cosine(query_embed, value)
                if sim>0.6:
                    distances[i] = distances.get(i, 0) + sim
    return distances, idx
    #return task_questions[idx][sorted(distances.items(), key=lambda x:x[0],reverse=True)[0][0]]

In [20]:
def sort_response(result,user_i):
    if len(result)<1:
        response="Sorry i couldn't understand your question. Can you give more detail?"
        quest=''
    else:
        best=sorted(result.items(), key=lambda x:x[1], reverse=True)[0]
        #response=find_task_ques(best)
        #####
        distances, idx=check_distanceq(best,user_i)
        if len(distances)>0:
            items=task_questions[idx][sorted(distances.items(), key=lambda x:x[0],reverse=True)[0][0]]
            quest,response=find_task_Summary(items)
        else:
            response="Sorry i couldn't understand your question. Can you give more detail?"
            quest=''
    return quest,response

In [ ]:
ans=''
def display_option():
    def f(x):
        global ans
        ans=x
    display (Markdown('<span style="color: blue">Hello I want to help you about corona virus. \
    Please write what are you wondering for? If you want to quit please write bye</span>'))
    interact(f, x=['General','Corona']);
    return ans

In [ ]:
def make_conv(ans):
    if ans=='Corona':
        Stop=False
        display (Markdown('<span style="color: blue">Hello I want to help you about corona virus. Please write what are you wondering for? If you want to quit please write bye</span>'))
        while (Stop==False):
            user_i=input()
            if(user_i!='bye'):
                ques=embeddings.check_distance(user_i)
                quest, response=sort_response(ques,user_i)
                responsek=response
                display (Markdown(f'<span style="color: red">Do you want to ask this:{quest}?</span>'))
            else:
                response=End_conv()
                Stop=True
            if len(response.split('\n'))>1:
                for resp_p in response.split('\n'):
                    display(Markdown(f'<span style="color: blue">{resp_p}</span>'))
            else:
                display(Markdown(f'<span style="color: blue">{response}</span>'))
    return responsek

## General Side

In [21]:
DDIR='data'
corpus_f='friends-corpus'
corpuss='ConvAIandFriends'
utterancesPath='utterances.jsonl'
corpus_chatter='chatterbot'
Coca='coqa-train-v1.0.json'
convai=['data_volunteers.json','data_intermediate.json','data_tolokers.json']

In [22]:
chatter_paths=os.listdir(os.path.join(DDIR, corpus_chatter))

In [23]:
path_f=path=os.path.join(DDIR, corpus_f,utterancesPath)

paths=[]
for conv in convai:
    paths.append(os.path.join(DDIR, conv))

In [24]:
paths

['data\\data_volunteers.json',
 'data\\data_intermediate.json',
 'data\\data_tolokers.json']

In [25]:
utterancesc=[]
for path in paths:
    with open(path, 'r', encoding='utf-8') as p:
        for line in p:
            utterancesc.append(json.loads(line))

In [26]:
len(utterancesc[2])

3127

In [27]:
delimiter='\t'
textlist =[]
for utterance in utterancesc:
    for utter in utterance:
        if len(utter['dialog'])>1:
            oldsender=utter['dialog'][0]['sender_class']
            for i in range(1,len(utter['dialog'])):
                sender=utter['dialog'][i]['sender_class']
                if sender!=oldsender:
                    text=utter['dialog'][i-1]['text']+delimiter+utter['dialog'][i]['text']
                oldsender=utter['dialog'][i]['sender_class']
                textlist.append(text)
            

In [28]:
utterances=[]
with open(path_f, 'r', encoding='utf-8') as p:
    for line in p:
        utterances.append(json.loads(line))

In [29]:
idtext={}
for ut in utterances:
    idtext[ut['id']]=ut['text']

In [30]:
text=''
delimiter='\t'
#textlist=[]
for ut in utterances:
    text=''
    if ut['reply-to'] != None:
        if ut['text']!='' and idtext[ut['reply-to']]!='':
            text +=idtext[ut['reply-to']]+delimiter+ut['text']
            textlist.append(text)

In [33]:
for c_path in chatter_paths:
    file=os.path.join(DDIR, corpus_chatter, c_path)
    with open(file, 'r', encoding='utf-8') as c:
            utt=yaml.safe_load(c)
    if c_path=='literature.yml':
        utt['conversations'][-2][1]=utt['conversations'][-2][1].replace('Tolkein','Tolkien')
    for convers in utt['conversations']:
        if type(convers[1])==dict:
            key=list(convers[1].keys())[0]
            value=list(convers[1].values())[0]
            text1=convers[0]+delimiter+key
            text2=convers[0]+delimiter+value
            textlist.append(text1)
            textlist.append(text2)
        else:
            text=convers[0]+delimiter+convers[1]
            textlist.append(text)

In [32]:
# data_c='corona'
# files=os.listdir(os.path.join(DDIR, data_c))
# dicts=[]
# summaries=[]
# for file in files:
#     if file.endswith('.dic'):
#         dicts.append(os.path.join(DDIR, data_c, file))
#     if file.endswith('.SUM'):
#         summaries.append(os.path.join(DDIR, data_c, file))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\corona'

In [34]:
# def merge_files(paths):
#     all_files={}
#     for path in paths:
#         dicts=joblib.load(path)
#         all_files={**all_files, **dicts}
#     return all_files

In [35]:
# all_dicts=merge_files(dicts)

NameError: name 'dicts' is not defined

In [16]:
# def create_Pairs(data, textlist,delimiter):
#     keys=list(data.keys())
#     #textlist=[]
#     for i, value in enumerate(data.values()):
#         for j in range(5):
#             ans=value.loc[j,'answer_sent']
#             ans=ans.strip()
#             if ans!='':
#                 pp=[': medrxiv',': biorxiv']
#                 startswith=['.',',',' .',' ,']
#                 ind1=[x for x in startswith if ans.startswith(x)]
#                 ind=[(ans.find(x), len(x)) for x in pp if ans.find(x)!=-1]
#                 if ind:
#                     ans=ans[ind[0][0]+ind[0][1]+1:]
#                 if ind1:
#                     ans=ans[1:]
#                 text=keys[i]+delimiter+ans
#                 textlist.append(text)
#     return textlist

In [17]:
#textlist=create_Pairs(all_dicts, textlist, delimiter)

In [36]:
PAD=0
SOS=1
EOS=2
class Vocab(object):
    def __init__(self, name):
        self.name=name
        self.word2idx={}
        self.wordcount={}
        self.idx2word={PAD:'PAD',EOS:'EOS', SOS:'SOS'}
        self.numberofWord=3
    
    def add_Sentence(self,sentence):
        for word in sentence.split(' '):
            self.check_Word(word)
    
    def check_Word(self,word):
        if word not in self.word2idx:
            self.wordcount[word]=1
            self.word2idx[word]=self.numberofWord
            self.idx2word[self.numberofWord]=word
            self.numberofWord +=1
        else:
            self.wordcount[word] +=1

In [37]:
def sentenceOperation(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def sentenceOperationC(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [39]:
MAX_LENGTH = 10
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def read_voc(data, name):
    pairs=[[sentenceOperation(sentence) for sentence in sentences.split('\t')]for sentences in data]
    voc=Vocab(name)
    return pairs,voc

def read_vocCor(data, name):
    pairs=[[sentenceOperationC(sentence) for sentence in sentences.split('\t')]for sentences in data]
    voc=Vocab(name)
    return pairs,voc

In [40]:
# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [41]:
def load_data(name):
    print('Starting')
    pairs,vocab=read_voc(textlist, name)
    print(f'Number of pairs is {len(pairs)}')
    pairs=filterPairs(pairs)
    print(f'After filters Number of pairs is {len(pairs)}')
    for pair in pairs:
        vocab.add_Sentence(pair[0])
        vocab.add_Sentence(pair[1])
    print("Counted words:", vocab.numberofWord)
    return vocab, pairs

In [42]:
voc, pairs = load_data(corpuss)

Starting
Number of pairs is 110191
After filters Number of pairs is 42690
Counted words: 8451


In [43]:
def indexesFromSentence(voc, sentence):
    return [voc.word2idx[word] for word in sentence.split(' ')] + [EOS]


def zeroPadding(l, fillvalue=PAD):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  13,   84,   21,   68,   13],
        [  29,   85,  122,   49,   40],
        [  81,   40,   49,    8,  758],
        [  31,   52,    8,   40,    2],
        [1072,   53,   69,  104,    0],
        [  12,   37,    9,  420,    0],
        [   8,   86,    2,    2,    0],
        [   9,   18,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([9, 9, 7, 7, 4])
target_variable: tensor([[ 49, 171,  13,  93,  13],
        [  8,   2,  40,   7,  29],
        [ 57,   0, 104,   8,  52],
        [826,   0,  69,   9, 761],
        [  9,   0, 121,   2,  37],
        [  2,   0, 122,   0, 346],
        [  0,   0,  18,   0,  18],
        [  0,   0,   2,   0,   2]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1]], dtype=torch.uint8)
max_target_len: 8


In [44]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [45]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [46]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size*2 , hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [47]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [48]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [49]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [50]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [58]:
def check_words(inputs):
    inputs=inputs.split(' ')
    for inputa in inputs:
        if inputa in Related_word:
            return True
    return False

In [63]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.idx2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            res=check_words(input_sentence)
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break

            
            if res:
                if(input_sentence!='bye'):
                    user_i=input_sentence
                    ques=embeddings.check_distance(user_i)
                    quest, response=sort_response(ques,user_i)
                    responsek=response
                    display (Markdown(f'<span style="color: red">Do you want to ask this:{quest}?</span>'))
                else:
                    response=End_conv()
                    Stop=True
                if len(response.split('\n'))>1:
                    for resp_p in response.split('\n'):
                        display(Markdown(f'<span style="color: blue">{resp_p}</span>'))
                else:
                    display(Markdown(f'<span style="color: blue">{response}</span>'))
                        # Normalize sentence
            else:
                input_sentence = sentenceOperation(input_sentence)
                # Evaluate sentence
                output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
                # Format and print response sentence
                output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
                print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [53]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 600
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.2
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 10000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.numberofWord, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.numberofWord, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [55]:
# Configure training/optimization

save_dir = os.path.join("data", "save")
corpus_name=corpuss
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 20000
print_every = 100
save_every = 10000

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 100; Percent complete: 0.5%; Average loss: 4.9232
Iteration: 200; Percent complete: 1.0%; Average loss: 3.8630
Iteration: 300; Percent complete: 1.5%; Average loss: 3.5967
Iteration: 400; Percent complete: 2.0%; Average loss: 3.3999
Iteration: 500; Percent complete: 2.5%; Average loss: 3.2856
Iteration: 600; Percent complete: 3.0%; Average loss: 3.1633
Iteration: 700; Percent complete: 3.5%; Average loss: 3.0633
Iteration: 800; Percent complete: 4.0%; Average loss: 3.0164
Iteration: 900; Percent complete: 4.5%; Average loss: 2.9416
Iteration: 1000; Percent complete: 5.0%; Average loss: 2.9305
Iteration: 1100; Percent complete: 5.5%; Average loss: 2.8434
Iteration: 1200; Percent complete: 6.0%; Average loss: 2.8082
Iteration: 1300; Percent complete: 6.5%; Average loss: 2.7543
Iteration: 1400; Percent complete: 7.0%; Average loss: 2.6922
Iteration: 1500; Percent complete: 7.5%; Average loss: 2.6344
Iterati

In [56]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
# evaluateInput(encoder, decoder, searcher, voc)

In [74]:
evaluateInput(encoder, decoder, searcher, voc)

> hey
Bot: hi how are you doing ? the wedding ?
> good. You
Bot: good . . .
> OK
Bot:  . .
> ok
Bot:  . .
> corona


<span style="color: red">Do you want to ask this:?</span>

<span style="color: blue">Sorry i couldn't understand your question. Can you give more detail?</span>

> Does corona dangerous


<span style="color: red">Do you want to ask this:What have we learned about infection prevention and control?</span>

...

> country
Bot: hello . how are you today ? ?
> which country affacted most by corona
Error: Encountered unknown word.
> which country affected most by corona


<span style="color: red">Do you want to ask this:Which country was less affected?</span>

<span style="color: blue">The summary of 3500 paper that we know is</span>

<span style="color: blue"> Covid-19 has spread to 48 countries worldwide, especily in iran, korea and some european countries, even affected  the united states. Less is known about its impact on psychiatric care, particularly in  the  united states, 'flattening the curve'), whereas countries with limited response, such as  the United States, have experienced steeper growth rates in cases. Covid-19 has spread from china to 29 other countries or regions including south korea, japan, thailand, singapore, vinam, taiwan, nep, and the united states. The current 5 infection has spread outwards from ch China to 29 countries or Regions. The outbreak of coronavirus disease 2019 (covid-19) has become a serious public heth situation in china. The number of infected patients is increasing in other countries (especily  in south korea, japan, and ity) The epidemic continues to spread throughout china at an arming rate. A totof 81300 cases were confirmed in china so far (march 20, 2020) and increasingly more cases have been confirmed abroad. The united kingdom, france and germany are at the highest risk, followed by ity, spain and  the nherlands. Other asian, african and south american countries are less affected till the beginning of april 2020. 77 and 861 cases were identified  in south korea and japan, respectively. except vinam and macau sar (which had 16 and 10 covid-19 cases, respectively), the other eight countries had a totof ≤3 cases. It first appeared in wuhan, china, but has now disseminated to multiple countries in the world, including  the united states.</span>

> heart
Bot: i am a little bit of a person
> heart disaese
Error: Encountered unknown word.
> q


In [ ]:
##Result of GRU Encoder and Decoder with 10000 iter Dropout=0.1
evaluateInput(encoder, decoder, searcher, voc)

In [ ]:
##Result of LSTM Encoder and Decoder with 10000 iter Dropout=0.1
evaluateInput(encoder, decoder, searcher, voc)

In [ ]:
evaluateInput(encoder, decoder, searcher, voc)

In [ ]:
evaluateInput(encoder, decoder, searcher, voc)